# Scattering Tutorial

* In this tutorial, you will analyze a data set of pulsar J1913+11025 obtained through the PALFA survey. 


* First, we import the required libraries and packages.


In [ ]:
%matplotlib notebook
import psrchive
import numpy as np
from scipy.optimize import curve_fit
import matplotlib.pyplot as plt
from matplotlib import gridspec

* The path to the data set is `/nanograv/notebook/homes/jupyterhub-benetgeperera/tutorials/bands_all.F4.Tp`.

* We can load and read in the data set using the PSRCHIVE package (http://psrchive.sourceforge.net). PSRCHIVE is a standard pulsar data analyzing package.

In [ ]:
psr = 'J1913+11025'
period = 0.92381106789333*1000.    #in ms

fn = 'bands_all.F4.Tp'
obs = psrchive.Archive_load(fn)
obs.dedisperse()
obs.pscrunch()
obs.remove_baseline()
data0 = obs.get_data()
freq = obs.get_frequencies()
nchan = data0.shape[2]
nbins = data0.shape[3]
if nchan > 1:
    weights = obs.get_weights()
    for ww in range(nbins):
        data0[:,0,:,ww] *= weights
data = data0[0,0,:,:]
nbands = 3    #ignore the lowest-freq band due to low S/N    

#print the shape of the data
print('Shape of the data array:')
data.shape


* The data set has 4 frequency bands. Let's plot pulse profiles of all frequency bands!


In [ ]:
#normalize the peak of profiles
for i in range(nchan):
    data[i,:] /= np.amax(data[i,:])

#center the peak of the profile
prof = np.mean(data, axis=0)
shift = int(nbins/2. - np.argmax(prof))
prof = np.roll(prof, shift)
data = np.roll(data, shift, axis=1)

xbins = np.arange(nbins)
xmin = 0
xmax = 1
ymin = -0.2
ymax = 1.2
plt.figure(figsize=(5,6))

for i in range(nbands):
    plt.subplot(nbands,1,i+1)
    if i < nbands-1:
        plt.tick_params(direction='in',axis='both',length=3,top=True,right=True,labelbottom=False,labelsize='large')
    else:
        plt.tick_params(direction='in',axis='both',length=3,top=True,right=True,labelbottom=True,labelsize='large')
        plt.xlabel('Pulse phase', fontsize='large')
    plt.xlim(xmin,xmax)
    plt.ylim(ymin, ymax)
    plt.plot(xbins/nbins,data[i,:],color='gray',linewidth=.5)
    plt.text(1.02*xmin+0.04,ymax*.75, r'$\nu$ ='+str(int(np.round(freq[i])))+' MHz',fontsize='large')


## Compute the scattering time ($\tau$) and scattering spectral index ($\alpha$). 

* We fit a Gaussian function convolve with a pulse-broadening function. We assume that the radio waves are scattered isotropically by a single thin scattering screen.


* The pulse-broadening function is $\exp({-1/\tau})/\tau$ and then $\tau(\nu) \propto \nu^{-\alpha}$.


* Let's fit a Gaussian convolved with a one-sided exponential function to the observed freq-band profiles.


* Then obtain the scattering time $\tau$ and scattering spectral index $\alpha$.


In [ ]:
def gauss_exp_model_mod2(x,*pp):
    tmp_prof = np.zeros((nbands,nbins))
    for j in range(nbands):
        gs = pp[j*3]*np.exp(-(np.arange(nbins)-pp[j*3+1])**2./(2.*pp[j*3+2]**2))
        tau = pp[len(pp)-2]*((freq[j]/1000.0)**(-1.*pp[len(pp)-1]))
        st = np.exp(-np.arange(nbins)/tau)/tau
        prf_st = np.convolve(gs,st,mode='full')
        tmp_prof[j,:] = prf_st[0:nbins]
    model = tmp_prof.flatten()
    return model

allprofs = data[0:nbands].flatten()
allx = np.arange(len(allprofs))

#get the bin of the peak of the highest freq band profile
mbin = np.argmax(data[0,:])

#initial guesses for the fit
tau_ini = 30.
spec_ini = 4.
pp = [.1, mbin, 10., .1, mbin, 10., .1,mbin,10., .1,mbin,10.,.1,mbin,10., tau_ini, spec_ini]
popt, pcov = curve_fit(gauss_exp_model_mod2,allx,allprofs,p0=pp,maxfev=100000,bounds=(0,1000))
perr = np.sqrt(np.diag(pcov))

tau = popt[len(popt)-2]*period/nbins
tau_err = perr[len(popt)-2]*period/nbins

print('\n*****************\n')
print('Tau (ms) = ', tau, '+/-', tau_err, ' at Freq 1 GHz')
spec = -1.*popt[len(popt)-1]
spec_err = perr[len(popt)-1]


print('Spec :', spec, '+/-', spec_err)
print('\n*****************\n')



* Let's overplot the Gaussian fits with the observed pulse profiles!


In [ ]:
gg = gauss_exp_model_mod2(allx,*popt)
profiles = np.resize(allprofs, (nbands,nbins))
templates = np.resize(gg, (nbands,nbins))

plt.figure(figsize=(5,8))

for i in range(nbands):
    plt.subplot(nbands,1,i+1)
    if i == 0: plt.title(psr+'\n'+ r'$\alpha_{sc}$ = '+ str(np.round(-1*spec,1))+
                         r'$\pm$'+str(np.round(spec_err,1))+r'       $\tau_{sc}$ = '+
                         str(int(np.round(tau)))+r'$\pm$'+str(int(np.round(tau_err)))+' ms', fontsize='large')        
    if i < nbands-1:
        plt.tick_params(direction='in',axis='both',length=3,top=True,right=True,labelbottom=False,labelsize='large')
    else:
        plt.tick_params(direction='in',axis='both',length=3,top=True,right=True,labelbottom=True,labelsize='large')
        plt.xlabel('Pulse phase', fontsize='large')
    plt.xlim(xmin,xmax)
    plt.ylim(ymin, ymax)
    fac = 1./np.amax(templates[i,:])
    plt.plot(xbins/nbins,profiles[i,:]*fac,color='gray',linewidth=.5)
    plt.plot(xbins/nbins,templates[i,:]*fac,color='k')
    plt.text(1.02*xmin+0.04,ymax*.75, r'$\nu$ ='+str(int(np.round(freq[i])))+' MHz',fontsize='large')
    tauf = tau*((freq[i]/1000.0)**spec)
    tauf_err = tau_err*((freq[i]/1000.0)**spec)
    plt.text(1.02*xmin+0.04,ymax*.55,r'$\tau$ = '+str(np.round(tauf,1))+r'$\pm$'+
             str(np.round(tauf_err,1))+' ms',fontsize='large')